**Полезные ссылки:**

https://habr.com/ru/articles/710016/

# **Попытка распознавания объектов благоустройства**
# **с помощью предобученной YOLOv8**

In [ ]:
# @title Установка библиотек

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/107.7 GB disk)


# **Сервисные функции**

In [ ]:
import time

# @title Контекстный менеджер для измерения времени операций
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # Расчет времени выполнения
        result_time = time.time()-self.t
        hour = int(result_time//3600)
        min = int(result_time//60)-hour*60
        sec = int(round(result_time%60))
        msec = round(1000*result_time%60)

        if hour > 0:
          print('\nВремя обработки: ' + str(hour)+' час. ' + str(min)+' мин.')
        elif min > 0:
          print('\nВремя обработки: ' + str(min)+' мин. ' + str(sec)+' сек.')
        elif sec > 0:
          print('\nВремя обработки: ' + str(sec)+' сек.')
        else:
          print('\nВремя обработки: ' + str(msec)+' мс.')


In [ ]:
import gdown

# @title Функция загрузки кадра из видео в Google Colab по указанному времени
def load_video(video_url, video_name):
    # Стартовая отметка времени
    t_0 = time.time()

    # Загружаем видео в Colab
    gdown.download(video_url, None, quiet=True)

    video_path = f"/content/{video_name}"

    # Расчет времени выполнения
    t_f = time.time()-t_0

    # Расчет времени обработи и приведение его к легко читаемому виду
    time_text = str(int(round(t_f//60,0)))+' мин '+str(int(round(t_f%60,0)))+' сек.'

    # Подведение итогов обработки
    print('Загрузка видеофайла: '+ video_name + '\nсделана за: ' + time_text)

    return video_path


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
from pathlib import Path

# @title Функция извлечения кадра из видео по указанному времени
def extract_frame_from_video(video_path, minutes, seconds, path_output):
    # Стартовая отметка времени
    t_0 = time.time()

    # Открываем видеофайл
    cap = cv2.VideoCapture(video_path)

    # Проверяем, успешно ли открыт файл
    if not cap.isOpened():
        print(f"Не удалось открыть видеофайл: {video_path}")
        return None

    # Рассчитываем время в кадрах
    target_frame_time = int((minutes * 60 + seconds) * cap.get(cv2.CAP_PROP_FPS))

    # Устанавливаем позицию в видео на нужный кадр
    cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_time)

    # Считываем кадр
    ret, frame = cap.read()

    # Отключаем видеопоток
    cap.release()

    # Проверяем, успешно ли считан кадр
    if not ret:
        print(f"Не удалось считать кадр для времени {minutes} минут {seconds} секунд.")
        return None

    # Расчет времени выполнения
    t_f = time.time()-t_0

    # Расчет времени обработи и приведение его к легко читаемому виду
    time_text = str(int(round(t_f//60,0)))+' мин '+str(int(round(t_f%60,0)))+' сек.'

    # Подведение итогов обработки
    print(f'Из видеофайла: {video_name}\nизвлечен кадр для времени: {minutes} мин. {seconds} сек. \nза: {time_text}\n')

    # Создаем папку Image, если она не существует
    Path(path_output).mkdir(parents=True, exist_ok=True)

    # Формируем имя файла на основе времени и сохраняем в папке Image
    file_name = f"frame_{minutes}min_{seconds}sec.png"
    file_path = Path(path_output) / file_name
    # cv2.imwrite(str(file_path), cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cv2.imwrite(str(file_path), frame)  # Убрали cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Выводим информацию о сохраненном файле
    print(f"Сохранен файл: {file_path}")
    print(f"Разрешение файла: {frame.shape[1]}x{frame.shape[0]} пикселей\n")

    return frame


In [ ]:
# @title Функция изменения ширины кадра (высота пропорционально)
def show_resized_image(image, new_width):
    """
    Отображает изображение с установленной новой шириной, сохраняя пропорции.

    Параметры:
    - image: numpy.ndarray, входное изображение.
    - new_width: int, новая ширина изображения.
    """
    aspect_ratio = image.shape[1] / image.shape[0]
    new_height = int(new_width / aspect_ratio)
    resized_image = cv2.resize(image, (new_width, new_height))
    cv2_imshow(resized_image)


# **Загрузка видео в Google Colab**

In [ ]:
# @title Загрузка файла...
video_url = 'https://drive.google.com/uc?id=1Kk5fx8C4nCe9Drxl-wTMP5MfjU4H1F0F'
video_name = 'DJI_0002 (1).MP4'

# Загрузка видео
video_path = load_video(video_url, video_name)


Загрузка видеофайла: DJI_0002 (1).MP4
сделана за: 0 мин 40 сек.


# **Извлечение кадров из видео-файла**

In [ ]:
# @title Извлечение кадра с 1 минуты 53 секунды
frame_01 = extract_frame_from_video(video_path, minutes=1, seconds=53, path_output='/content/Image')

# Вывод изображения
if frame_01 is not None:
    show_resized_image(frame_01, new_width=1500)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Извлечение кадра с 2 минуты 05 секунд
frame_02 = extract_frame_from_video(video_path, minutes=2, seconds=5, path_output='/content/Image')

# Вывод изображения
if frame_02 is not None:
    show_resized_image(frame_02, new_width=1500)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Извлечение кадра с 2 минуты 17 секунд
frame_03 = extract_frame_from_video(video_path, minutes=2, seconds=17, path_output='/content/Image')

# Вывод изображения
if frame_03 is not None:
    show_resized_image(frame_03, new_width=1500)

Output hidden; open in https://colab.research.google.com to view.

# Модель YOLOv8 Nano

In [ ]:
from ultralytics import YOLO
import os
from pathlib import Path

from IPython.display import display, Image


In [ ]:
from ultralytics import YOLO
model_nano = YOLO('yolov8n.pt')


100%|██████████| 6.23M/6.23M [00:00<00:00, 162MB/s]


In [ ]:
# @title Создание каталога для сохранения результата
Results_Path = 'Results'

if not os.path.isdir(Results_Path):
  os.mkdir(Results_Path)


In [ ]:
# @title Обработка извлеченых кадров
with timex():
  results = model_nano(source=frame_01, save=True, show=False, project=Results_Path)
  results = model_nano(source=frame_02, save=True, show=False, project=Results_Path)
  results = model_nano(source=frame_03, save=True, show=False, project=Results_Path)


0: 384x640 (no detections), 298.2ms
Speed: 8.5ms preprocess, 298.2ms inference, 16.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to Results/predict

0: 384x640 4 cars, 138.7ms
Speed: 5.3ms preprocess, 138.7ms inference, 15.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to Results/predict2

0: 384x640 1 car, 137.6ms
Speed: 4.1ms preprocess, 137.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to Results/predict3

Время обработки: 3 сек.


## **Вывод: Результат не удовлетворительный**

# Модель YOLOv8 Extra Large

In [ ]:
# @title Загрузка самой тяжелой модели Extra Large
model_X = YOLO('yolov8x.pt')


100%|██████████| 131M/131M [00:00<00:00, 243MB/s]


In [ ]:
# @title Обработка извлеченых кадров
with timex():
  results = model_X(source=frame_01, save=True, show=False, project=Results_Path)
  results = model_X(source=frame_02, save=True, show=False, project=Results_Path)
  results = model_X(source=frame_03, save=True, show=False, project=Results_Path)



0: 384x640 11 cars, 7625.2ms
Speed: 3.7ms preprocess, 7625.2ms inference, 11.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to Results/predict4

0: 384x640 1 person, 13 cars, 2 trucks, 5616.9ms
Speed: 3.7ms preprocess, 5616.9ms inference, 10.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to Results/predict5

0: 384x640 5 cars, 1 bus, 3 trucks, 7852.2ms
Speed: 3.9ms preprocess, 7852.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to Results/predict6

Время обработки: 25 сек.


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# Список путей к каталогам
paths_to_predict = [
    '/content/Results/predict4',
    '/content/Results/predict5',
    '/content/Results/predict6'
]

# @title Отображение изображений из каталогов, где результаты распознавания
for path_to_predict in paths_to_predict:
    # Получаем список файлов в текущем каталоге
    files = [file for file in os.listdir(path_to_predict) if file.endswith(('.png', '.jpg', '.jpeg'))]

    # Отображаем изображения в цикле
    for file in files:
        img_path = os.path.join(path_to_predict, file)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # Отображаем изображение с использованием matplotlib
        plt.figure(figsize=(20, 12))
        plt.axis('off')
        plt.title(file)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()


Output hidden; open in https://colab.research.google.com to view.

## **Вывод: Результат получше, есть некоторые распознанные объекты,**
## **но объекты благоустройства не распознаны вообще...**

# Деление кадров на фрагменты

Поделим каждый кадр на 16 частей и распознаем их по отдельности.

YOLO8 принимает на вход кадры 640х384, поэтому при исходном изображении 3840х2160 деление на 16 частей как раз будет примерно соответствовать этому разрешению.

# **Эксперимент 1 (frame_1min_53sec)**

In [ ]:
# Выбираем файл для нарезки
file_for_crop = '/content/Image/frame_1min_53sec.png'

# Каталог для сохранения результата
Cropped_Path = 'Cropped'

# Создание каталога для сохранения результата
if not os.path.isdir(Cropped_Path):
  os.mkdir(Cropped_Path)

# Загрузка изображения
img = cv2.imread(file_for_crop, cv2.IMREAD_COLOR)

# Извлечение имени файла
_, pic_name = os.path.split(file_for_crop)
print(f'Файл {pic_name} c разрешением {img.shape[1]}x{img.shape[0]}')

# Выводим изображения для проверки

# @title Деление кадра **frame_1min_53sec** на фрагменты
for scale in [1,4]:
  plt.figure(figsize = (20,12))
  subplot_num = 0
  width = img.shape[1]//scale
  height =  img.shape[0]//scale

  # Создаю и заполняю полотно
  for i in range(scale):
    for j in range(scale):
      subplot_num += 1
      sub_img = img[i*height:(i+1)*height,j*width:(j+1)*width,::-1]
      sub_name = str(width) + 'x' + str(height) + '_part_'+str(subplot_num).rjust(2, '0')
      # Размещаем фрагмент изображение на соответствущем подграфике
      ax = plt.subplot(scale,scale,subplot_num)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      ax.set_title(sub_name)
      plt.imshow(sub_img)
      # Сохраняем фрагмент в папку
      sub_file_name = pic_name[:-4] + '_' + sub_name +'.png'
      cv2.imwrite(os.path.join(Cropped_Path,sub_file_name), sub_img[:,:,::-1])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Обработка и обнаружение объектов на изображениях в папке
with timex():
  results = model_X(source=Cropped_Path, save=True, show=False, project=Results_Path)



image 1/17 /content/Cropped/frame_1min_53sec_3840x2160_part_01.png: 384x640 11 cars, 2809.4ms
image 2/17 /content/Cropped/frame_1min_53sec_960x540_part_01.png: 384x640 (no detections), 3270.5ms
image 3/17 /content/Cropped/frame_1min_53sec_960x540_part_02.png: 384x640 1 person, 24 cars, 2 trucks, 3879.8ms
image 4/17 /content/Cropped/frame_1min_53sec_960x540_part_03.png: 384x640 8 cars, 3 trucks, 2606.1ms
image 5/17 /content/Cropped/frame_1min_53sec_960x540_part_04.png: 384x640 (no detections), 2840.6ms
image 6/17 /content/Cropped/frame_1min_53sec_960x540_part_05.png: 384x640 2 persons, 6 cars, 1 truck, 2631.5ms
image 7/17 /content/Cropped/frame_1min_53sec_960x540_part_06.png: 384x640 1 car, 2 benchs, 5 chairs, 3667.1ms
image 8/17 /content/Cropped/frame_1min_53sec_960x540_part_07.png: 384x640 (no detections), 3342.2ms
image 9/17 /content/Cropped/frame_1min_53sec_960x540_part_08.png: 384x640 1 book, 2624.5ms
image 10/17 /content/Cropped/frame_1min_53sec_960x540_part_09.png: 384x640 3 car

In [ ]:
# Выбираем обработанный файл
file_for_crop = '/content/Image/frame_1min_53sec.png'

# Выбираем каталог куда был сохранен предикт
Predict_Dir = '/content/Results/predict7'

# Извлечение имени файла
_, pic_name = os.path.split(file_for_crop)
print(f'Файл {pic_name} c разрешением {img.shape[1]}x{img.shape[0]}')

# Выводим изображения для проверки

# @title Визуализация результата обнаружения объектов
for scale in [1,4]:
  plt.figure(figsize = (20,12))
  subplot_num = 0
  width = img.shape[1]//scale
  height =  img.shape[0]//scale

  # Создаю и заполняю полотно
  for i in range(scale):
    for j in range(scale):
      subplot_num += 1
      sub_img = img[i*height:(i+1)*height,j*width:(j+1)*width,::-1]             #.rjust(2, '0')
      sub_name = str(width) + 'x' + str(height) + '_part_'+str(subplot_num).rjust(2, '0')
      sub_file_name = pic_name[:-4] + '_' + sub_name +'.png'
      sub_img = cv2.imread(os.path.join(Predict_Dir,sub_file_name), cv2.IMREAD_COLOR)
      # Размещаем фрагмент изображение на соответствущем подграфике
      ax = plt.subplot(scale,scale,subplot_num)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      ax.set_title(sub_name)
      plt.imshow(sub_img[:,:,::-1])


Output hidden; open in https://colab.research.google.com to view.

## **Вывод:**
## **с объектами благоустройства ошиблась...**
Вроде бы нашла скамейки и стулья, но, к сожалению, ошиблась. Там другой инвентарь для детей.

А вот реальные скамейки возле дома не увидела.

Распознала большую часть машин, грузовиков, но были ошибки

Распознала людей, даже двоих по отдельности, которые шли рядом, а также человека с собакой (без собаки)

# **Эксперимент 2 (frame_2min_5sec)**

In [ ]:
# Выбираем файл для нарезки
file_for_crop = '/content/Image/frame_2min_5sec.png'

# Каталог для сохранения результата
Cropped_Path = 'Cropped2'

# Создание каталога для сохранения результата
if not os.path.isdir(Cropped_Path):
  os.mkdir(Cropped_Path)

# Загрузка изображения
img = cv2.imread(file_for_crop, cv2.IMREAD_COLOR)

# Извлечение имени файла
_, pic_name = os.path.split(file_for_crop)
print(f'Файл {pic_name} c разрешением {img.shape[1]}x{img.shape[0]}')

# Выводим изображения для проверки

# @title Деление кадра **frame_2min_5sec** на фрагменты
for scale in [1,4]:
  plt.figure(figsize = (20,12))
  subplot_num = 0
  width = img.shape[1]//scale
  height =  img.shape[0]//scale

  # Создаю и заполняю полотно
  for i in range(scale):
    for j in range(scale):
      subplot_num += 1
      sub_img = img[i*height:(i+1)*height,j*width:(j+1)*width,::-1]
      sub_name = str(width) + 'x' + str(height) + '_part_'+str(subplot_num).rjust(2, '0')
      # Размещаем фрагмент изображение на соответствущем подграфике
      ax = plt.subplot(scale,scale,subplot_num)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      ax.set_title(sub_name)
      plt.imshow(sub_img)
      # Сохраняем фрагмент в папку
      sub_file_name = pic_name[:-4] + '_' + sub_name +'.png'
      cv2.imwrite(os.path.join(Cropped_Path,sub_file_name), sub_img[:,:,::-1])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Обработка и обнаружение объектов на изображениях в папке
with timex():
  results = model_X(source=Cropped_Path, save=True, show=False, project=Results_Path)



image 1/17 /content/Cropped2/frame_2min_5sec_3840x2160_part_01.png: 384x640 1 person, 13 cars, 2 trucks, 2739.1ms
image 2/17 /content/Cropped2/frame_2min_5sec_960x540_part_01.png: 384x640 (no detections), 3081.6ms
image 3/17 /content/Cropped2/frame_2min_5sec_960x540_part_02.png: 384x640 6 cars, 1 bus, 2643.5ms
image 4/17 /content/Cropped2/frame_2min_5sec_960x540_part_03.png: 384x640 1 car, 1 truck, 3813.5ms
image 5/17 /content/Cropped2/frame_2min_5sec_960x540_part_04.png: 384x640 2 cars, 3357.6ms
image 6/17 /content/Cropped2/frame_2min_5sec_960x540_part_05.png: 384x640 8 cars, 2611.1ms
image 7/17 /content/Cropped2/frame_2min_5sec_960x540_part_06.png: 384x640 2 persons, 12 cars, 1 truck, 2604.8ms
image 8/17 /content/Cropped2/frame_2min_5sec_960x540_part_07.png: 384x640 1 person, 3 cars, 1 truck, 2637.9ms
image 9/17 /content/Cropped2/frame_2min_5sec_960x540_part_08.png: 384x640 1 person, 6 cars, 2 trucks, 4135.6ms
image 10/17 /content/Cropped2/frame_2min_5sec_960x540_part_09.png: 384x64

In [ ]:
# Выбираем обработанный файл
file_for_crop = '/content/Image/frame_2min_5sec.png'

# Выбираем каталог куда был сохранен предикт
Predict_Dir = '/content/Results/predict8'

# Извлечение имени файла
_, pic_name = os.path.split(file_for_crop)
print(f'Файл {pic_name} c разрешением {img.shape[1]}x{img.shape[0]}')

# Выводим изображения для проверки

# @title Визуализация результата обнаружения объектов
for scale in [1,4]:
  plt.figure(figsize = (20,12))
  subplot_num = 0
  width = img.shape[1]//scale
  height =  img.shape[0]//scale

  # Создаю и заполняю полотно
  for i in range(scale):
    for j in range(scale):
      subplot_num += 1
      sub_img = img[i*height:(i+1)*height,j*width:(j+1)*width,::-1]             #.rjust(2, '0')
      sub_name = str(width) + 'x' + str(height) + '_part_'+str(subplot_num).rjust(2, '0')
      sub_file_name = pic_name[:-4] + '_' + sub_name +'.png'
      sub_img = cv2.imread(os.path.join(Predict_Dir,sub_file_name), cv2.IMREAD_COLOR)
      # Размещаем фрагмент изображение на соответствущем подграфике
      ax = plt.subplot(scale,scale,subplot_num)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      ax.set_title(sub_name)
      plt.imshow(sub_img[:,:,::-1])


Output hidden; open in https://colab.research.google.com to view.

## **Вывод:**
В целом, обнаружение объектов и ошибки практически те же самые, что и на предыдущем рисунке...

# **Эксперимент 3 (frame_2min_17sec)**

In [ ]:
# Выбираем файл для нарезки
file_for_crop = '/content/Image/frame_2min_17sec.png'

# Каталог для сохранения результата
Cropped_Path = 'Cropped3'

# Создание каталога для сохранения результата
if not os.path.isdir(Cropped_Path):
  os.mkdir(Cropped_Path)

# Загрузка изображения
img = cv2.imread(file_for_crop, cv2.IMREAD_COLOR)

# Извлечение имени файла
_, pic_name = os.path.split(file_for_crop)
print(f'Файл {pic_name} c разрешением {img.shape[1]}x{img.shape[0]}')

# Выводим изображения для проверки

# @title Деление кадра **frame_2min_17sec** на фрагменты
for scale in [1,4]:
  plt.figure(figsize = (20,12))
  subplot_num = 0
  width = img.shape[1]//scale
  height =  img.shape[0]//scale

  # Создаю и заполняю полотно
  for i in range(scale):
    for j in range(scale):
      subplot_num += 1
      sub_img = img[i*height:(i+1)*height,j*width:(j+1)*width,::-1]
      sub_name = str(width) + 'x' + str(height) + '_part_'+str(subplot_num).rjust(2, '0')
      # Размещаем фрагмент изображение на соответствущем подграфике
      ax = plt.subplot(scale,scale,subplot_num)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      ax.set_title(sub_name)
      plt.imshow(sub_img)
      # Сохраняем фрагмент в папку
      sub_file_name = pic_name[:-4] + '_' + sub_name +'.png'
      cv2.imwrite(os.path.join(Cropped_Path,sub_file_name), sub_img[:,:,::-1])


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# @title Обработка и обнаружение объектов на изображениях в папке
with timex():
  results = model_X(source=Cropped_Path, save=True, show=False, project=Results_Path)



image 1/17 /content/Cropped3/frame_2min_17sec_3840x2160_part_01.png: 384x640 5 cars, 1 bus, 3 trucks, 4300.9ms
image 2/17 /content/Cropped3/frame_2min_17sec_960x540_part_01.png: 384x640 7 cars, 1 truck, 2610.1ms
image 3/17 /content/Cropped3/frame_2min_17sec_960x540_part_02.png: 384x640 1 person, 2 cars, 3 buss, 3 trucks, 2624.1ms
image 4/17 /content/Cropped3/frame_2min_17sec_960x540_part_03.png: 384x640 4 cars, 2634.4ms
image 5/17 /content/Cropped3/frame_2min_17sec_960x540_part_04.png: 384x640 (no detections), 3253.1ms
image 6/17 /content/Cropped3/frame_2min_17sec_960x540_part_05.png: 384x640 5 cars, 3873.6ms
image 7/17 /content/Cropped3/frame_2min_17sec_960x540_part_06.png: 384x640 1 person, 2 cars, 1 bench, 3 chairs, 2607.6ms
image 8/17 /content/Cropped3/frame_2min_17sec_960x540_part_07.png: 384x640 1 car, 2 boats, 1 bench, 1 chair, 2618.1ms
image 9/17 /content/Cropped3/frame_2min_17sec_960x540_part_08.png: 384x640 1 person, 2614.5ms
image 10/17 /content/Cropped3/frame_2min_17sec_96

In [ ]:
# Выбираем обработанный файл
file_for_crop = '/content/Image/frame_2min_17sec.png'

# Выбираем каталог куда был сохранен предикт
Predict_Dir = '/content/Results/predict9'

# Извлечение имени файла
_, pic_name = os.path.split(file_for_crop)
print(f'Файл {pic_name} c разрешением {img.shape[1]}x{img.shape[0]}')

# Выводим изображения для проверки

# @title Визуализация результата обнаружения объектов
for scale in [1,4]:
  plt.figure(figsize = (20,12))
  subplot_num = 0
  width = img.shape[1]//scale
  height =  img.shape[0]//scale

  # Создаю и заполняю полотно
  for i in range(scale):
    for j in range(scale):
      subplot_num += 1
      sub_img = img[i*height:(i+1)*height,j*width:(j+1)*width,::-1]             #.rjust(2, '0')
      sub_name = str(width) + 'x' + str(height) + '_part_'+str(subplot_num).rjust(2, '0')
      sub_file_name = pic_name[:-4] + '_' + sub_name +'.png'
      sub_img = cv2.imread(os.path.join(Predict_Dir,sub_file_name), cv2.IMREAD_COLOR)
      # Размещаем фрагмент изображение на соответствущем подграфике
      ax = plt.subplot(scale,scale,subplot_num)
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      ax.set_title(sub_name)
      plt.imshow(sub_img[:,:,::-1])


Output hidden; open in https://colab.research.google.com to view.

## **Вывод:**
## **с объектами благоустройства ошибается...**
Ура!!! Нашла одну реальную скамейку у мини-стадиона, но, там были и другие скамейки, которые не были найдены. Да и в других местах тоже их не нашла, которые были на территории детского учреждения и возле домов.

Распознала большую часть машин, грузовиков и даже автобусы, но были и ошибки.

По поводу людей... где-то распознала, а где-то и ошиблась, т.е. это был не человек, а скамейка :-)